## Тегирование нейронной части речи

Теперь мы собираемся решить ту же проблему тегирования POS с помощью нейронных сетей.
<img src=https://i.stack.imgur.com/6pdIT.png width=320>

С точки зрения глубокого обучения это задача прогнозирования последовательности выходных данных, выровненной с последовательностью входных данных. Есть несколько задач, соответствующих этой формулировке:
* Пометка части речи - вспомогательная задача для многих проблем НЛП
* Распознавание именованных объектов — для чат-ботов и поисковых роботов
* Предсказание структуры белка - для биоинформатики

In [ ]:
# %tensorflow_version 1.x

In [ ]:
import tensorflow as tf

In [ ]:
import nltk
import sys
import numpy as np

nltk.download('brown') # скачиваем аннотированный корпус POS-тегов
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal') # загружаем отмеченные предложения
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
<ipython-input-2-06938dfeccdb>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size=0.25,random_state=42)

In [ ]:
from IPython.display import HTML, display
def draw(sentence):
    words,tags = zip(*sentence)
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))


draw(data[11])
draw(data[10])
draw(data[7])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


NOUN,VERB,merger,proposed


### Построение словарей

Как и раньше, нам нужно построить отображение токенов на целочисленные идентификаторы. На этот раз наша модель работает на уровне слов, обрабатывая одно слово за шаг RNN. Это означает, что нам придется иметь дело с гораздо большим словарным запасом..

К счастью для нас, мы получаем эти слова только в качестве входных данных, то есть нам не нужно их предсказывать. Это означает, что мы можем иметь большой словарный запас бесплатно, используя встраивание слов.

In [ ]:
from collections import Counter
word_counts = Counter()
for sentence in data:
    words,tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

#let's measure what fraction of data words are in the dictionary
print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [ ]:
from collections import defaultdict
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

конвертировать слова и теги в матрицу фиксированного размера

In [ ]:
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Converts a list of names into rnn-digestable matrix with paddings added after the end"""

    max_len = max_len or max(map(len,lines))
    matrix = np.empty([len(lines), max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix



In [ ]:
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

### Построить модель

В отличие от нашей предыдущей лаборатории, на этот раз мы сосредоточимся на высокоуровневом интерфейсе keras для рекуррентных нейронных сетей. Это настолько просто, насколько вы можете получить с помощью RNN, хотя и несколько ограничено для сложных задач, таких как seq2seq.

По умолчанию все RNN keras применяются ко всей последовательности входных данных и создают последовательность скрытых состояний (return_sequences=True или только последнее скрытое состояние (return_sequences=False). Все повторения происходят под капотом.

В верхней части нашей модели нам нужно применить плотный слой к каждому временному шагу независимо. На данный момент keras.layers.Dense по умолчанию применяется один раз ко всем объединенным временным шагам.Мы используем keras.layers.TimeDistributed для изменения плотного слоя, чтобы он применялся как к пакетной, так и к временной осям.

In [ ]:
import keras
import keras.layers as L

model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))

#добавьте верхний слой, который предсказывает вероятности тегов
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          500100    
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 64)          7360      
                                                                 
 time_distributed (TimeDistr  (None, None, 14)         910       
 ibuted)                                                         
                                                                 
Total params: 508,370
Trainable params: 508,370
Non-trainable params: 0
_________________________________________________________________


__Обучение:__  в этом случае мы не хотим заранее готовить весь набор обучающих данных. Основная причина в том, что длина каждого пакета зависит от максимальной длины предложения в пакете. Это оставляет нам два варианта: использовать собственный обучающий код, как на предыдущем семинаре, или использовать генераторы.

В моделях Keras есть метод model.fit_generator, который принимает генератор Python, выдающий по одному пакету за раз. Но сначала нам нужно реализовать такой генератор:

In [ ]:
import keras
import keras.layers as L

In [ ]:
from keras.utils.np_utils import to_categorical
BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"

    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot


__Обратные вызовы:__  еще одна вещь, которая нам нужна, — это измерение производительности модели. Сложность заключается не в том, чтобы считать точность после окончания предложения (при заполнении), а в том, чтобы убедиться, что мы считаем все данные проверки ровно один раз.
Хотя нет ничего невозможного в том, чтобы убедить Keras сделать все это, мы можем также написать собственный обратный вызов, который сделает это.
Обратные вызовы Keras позволяют вам написать собственный код, который будет запускаться один раз в каждую эпоху или каждый мини-пакет. Мы определим его через LambdaCallback

In [ ]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    #predict tag probabilities of shape [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #compute accurary excluding padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()


In [ ]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


<ipython-input-16-e75a6e7fae6a>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.2479
Measuring validation accuracy...
448/448 [==============================] - 8s 18ms/step

Validation accuracy: 0.94102

1343/1343 [==============================] - 130s 93ms/step - loss: 0.2479
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0579
Measuring validation accuracy...
448/448 [==============================] - 8s 17ms/step

Validation accuracy: 0.94438

1343/1343 [==============================] - 102s 76ms/step - loss: 0.0579
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0513
Measuring validation accuracy...
448/448 [==============================] - 8s 17ms/step

Validation accuracy: 0.94664

1343/1343 [==============================] - 96s 72ms/step - loss: 0.0513
Epoch 4/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0467
Measuring validation accuracy...
448/448 [==============================] - 8s 17ms/step

Validation accuracy: 0.9

Измерьте окончательную точность на всем тестовом наборе.

In [ ]:
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras has gone on a rampage again, please contact course staff."

448/448 [==============================] - 7s 16ms/step
Final accuracy: 0.94693


### Двунаправленный

Поскольку мы анализируем полную последовательность, мы можем смотреть на будущие данные. Простой способ добиться этого — двигаться в обоих направлениях одновременно, создавая __двунаправленную RNN__.

В Keras вы можете добиться этого как вручную (используя два LSTM и Concatenate), так и используя __keras.layers.Bidirectional__.

Он работает так же, как TimeDistributed, который мы видели раньше: вы оборачиваете его вокруг рекуррентного слоя (сейчас SimpleRNN, а позже LSTM/GRU), и он фактически создает два слоя под капотом.

Ваша первая задача — использовать такой слой, как наш POS-тегер.

In [ ]:
#Определите модель, использующую двунаправленный SimpleRNN
model = keras.models.Sequential()

model.add(L.InputLayer([None], dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.Bidirectional(L.SimpleRNN(64, return_sequences = True)))

#добавьте верхний слой, который предсказывает вероятности тегов
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          500100    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        14720     
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, None, 14)         1806      
 ibuted)                                                         
                                                                 
Total params: 516,626
Trainable params: 516,626
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

<ipython-input-14-e75a6e7fae6a>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


Epoch 1/5
1344/1343 [==============================] - ETA: 0s - loss: 0.2052
Measuring validation accuracy...
448/448 [==============================] - 15s 33ms/step

Validation accuracy: 0.95479

1343/1343 [==============================] - 224s 162ms/step - loss: 0.2052
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0435
Measuring validation accuracy...
448/448 [==============================] - 13s 29ms/step

Validation accuracy: 0.96054

1343/1343 [==============================] - 161s 120ms/step - loss: 0.0435
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0356
Measuring validation accuracy...
448/448 [==============================] - 13s 29ms/step

Validation accuracy: 0.96230

1343/1343 [==============================] - 171s 127ms/step - loss: 0.0356
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0299
Measuring validation accuracy...
448/448 [==============================] - 14s 31ms/step

Valida

In [ ]:
acc = compute_test_accuracy(model)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 13s 30ms/step

Final accuracy: 0.96024
Well done!
